In [3]:
#import library
import os
import numpy as np
import scipy.io
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from mne.decoding import CSP
import mne
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

#filePath = '/Users/yamamotokouhei/subjectsData/data20221122'
#filePath = '/Users/yamamotokouhei/subjectsData/braindata20221003'
#labPC
filePath = '/Users/kyamamoto/Desktop/けんきゅうしつ/実験データ/20230208_zzikken'

brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']

# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65_label1 = list()#label1のデータ
o_data65_label2 = list()#label2のデータ
o_data = list()
label_data = list()#labelを結合したデータ
task1_data = list()
task2_data = list()

All_data = list()  # 全データを結合

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    #b = np.average(after_data)
    #c = np.var(after_data)
    return after_data

file_name = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name = sorted(file_name)
        file_name.append(file)
print('Filelist')
print(file_name)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kyamamoto/Desktop/けんきゅうしつ/実験データ/20230208_zzikken'

In [ ]:
#label1の設定
file_name1 = [file_name[9],file_name[10],file_name[11]]
file_num1 = len(file_name1)
#label2の設定
file_name2 = [file_name[12],file_name[13],file_name[14]]
file_num2 = len(file_name2)
# 変数など
trial_num = 30
all_trial = len(file_name1)+len(file_name2) * trial_num
all_d = 0
CH = 64
#label1を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label1.append(o_data)
#label2を読み込む
for s in range(file_num2):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name2[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label2.append(o_data)
print("data_loading")

: 

In [ ]:
c1 = o_data65_label1[0].eeg[: , 0:8190]
c2 = o_data65_label2[0].eeg[: , 0:8190]
for n in range(len(file_name1)*trial_num-1):  # all_trial
    g1 = o_data65_label1[n+1].eeg[: , 0:8190]
    g2 = o_data65_label2[n+1].eeg[: , 0:8190]
    c1 = np.dstack([c1, g1])
    c2 = np.dstack([c2, g2])


: 

In [ ]:
all_data = np.dstack([c1, c2])
all_data = all_data.transpose(2,0,1)
print(np.shape(all_data))
all_data = all_data.astype(np.double)

: 

In [ ]:
#ラベル生成
label = np.zeros(180,dtype=int)
label[90:] = 1
# a = list()
# b = list() 
# for n in range(len(file_name1)*trial_num):  # all_trial
#     label1 = o_data65_label1[n].label
#     label2 = o_data65_label2[n].label
#     if label1 == 2:
#         label1 = 0
#     if label2 == 2:
#         label2 = 0
#     a.append(label1)
#     b.append(label2)
# label = a+b

: 

In [ ]:
label = np.array(label)

: 

In [ ]:
print(label)

: 

In [ ]:
filter3 = signal.firwin(numtaps=51, cutoff=[5, 40], fs=1024, pass_zero=False)
for i in range(180):
    #all_data[i] -=np.mean(all_data[i][:][:])
    for j in range(64):
        all_data[i][j] = signal.lfilter(filter3, 1, all_data[i][j][:])
for i in range(180):
    all_data[i] -=np.mean(all_data[i][:][:])
    for j in range(64):
        all_data[i][j] = Standardization(all_data[i][j][:])



: 

In [ ]:
all_indices = list(range(180))
sub, main = train_test_split(all_indices, test_size=0.5,random_state=42)
#print(main)
train = all_data[sub,:,4*1024:]
test = all_data[main,:,4*1024:]
train_label = label[sub]
test_label = label[main]
print(np.shape(train))
print(np.shape(test))
cv = ShuffleSplit(5, test_size=0.2, random_state=42)
cv_split = cv.split(train)

: 

In [ ]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=2, log=None, reg=None)
# csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='csp_space')
# Use scikit-learn Pipeline with cross_val_score function

: 

In [ ]:
n_channels = 64
sampling_freq = 1024  # in Hertz
info = mne.create_info(ch_names=brainAdress, ch_types="eeg", sfreq=sampling_freq)
info.set_montage('biosemi64')


: 

In [ ]:
# Printing the results
class_balance = np.mean(train_label == train_label[0])
class_balance = max(class_balance, 1. - class_balance)
#print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),class_balance))

# plot CSP patterns estimated on full data for visualization
kenshou = csp.fit_transform(train, train_label)
csp.fit_transform(train, train_label)
csp.plot_patterns(info,ch_type='eeg', units='None', size=1.5)

: 

In [ ]:
print(np.shape(kenshou))
# plt.xlabel('trial')
plt.plot(kenshou)

: 

In [ ]:
basac = csp.transform(test)
print(np.shape(basac))
# plt.xlabel('trial')
# plt.plot(basac)

: 

In [ ]:
from sklearn.model_selection import cross_val_score
X_train, X_test, Y_train, Y_test = train_test_split(basac, test_label, test_size=0.4,random_state=42)
model =SVC(kernel='linear')
scores = cross_val_score(model, X_test, Y_test)
model.fit(X_train,Y_train)
print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

print('Cross-Validation scores: {}'.format(scores))
# スコアの平均値
import numpy as np
print('Average score: {}'.format(np.mean(scores)))

: 

In [ ]:
# # 境界線プロット用の格子状データを生成
# x1 = np.linspace(0.0, 10.0, 100)
# x2 = np.linspace(0.0, 10.0, 100)
# X1, X2 = np.meshgrid(x1, x2)    
# plot_X = np.c_[X1.ravel(), X2.ravel()]

# plot_y = model.predict(plot_X)

# # 検証結果の表示
# print("plot X：", plot_X)
# print("plot y：", plot_y)

# グラフ設定
plt.figure(figsize=(10, 10))
# # 格子データで散布図をプロットし、決定境界を描画（y=0:blue, y=1:red）
# plt.scatter(plot_X.T[0][plot_y <= 0], plot_X.T[1][plot_y <= 0], marker='o', color="blue", alpha=0.2)
# plt.scatter(plot_X.T[0][plot_y > 0], plot_X.T[1][plot_y > 0], marker='o', color="red", alpha=0.2)

# 学習用データを散布図にプロット（y=0:blue, y=1:red）
plt.scatter(X_train.T[0][Y_train <= 0], X_train.T[1][Y_train <= 0], marker='o', color="blue", alpha=1.0,label = "slow")
plt.scatter(X_train.T[0][Y_train > 0], X_train.T[1][Y_train > 0], marker='o', color="red", alpha=1.0,label = "fast")
plt.legend()
plt.xlabel('x1')            # x軸ラベル
plt.ylabel('x2')            # y軸ラベル
# plt.xlim(-4, 4)
# plt.ylim(-4, 4)
plt.show()


: 

In [ ]:
from mlxtend.plotting import plot_decision_regions
plot_decision_regions(X_train, Y_train, clf=model)
plt.xlabel('x1')            # x軸ラベル
plt.ylabel('x2') 
plt.xlim(-5, 4)
plt.ylim(-5, 2)
plt.show()

: 

In [ ]:
from mlxtend.plotting import plot_decision_regions
plot_decision_regions(X_test, Y_test, clf=model)
plt.xlabel('x1')            # x軸ラベル
plt.ylabel('x2') 
plt.xlim(-5, 4)
plt.ylim(-5, 2)
plt.show()


: 